In [1]:
import pandas as pd
import numpy as np


url = 'https://drive.google.com/uc?id=1lG4WZdJhHhvCR-a78ZvXBYB66gOKl47K&export=download'
df = pd.read_csv(url)
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [2]:
df.tail()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150
999,1000,2023-04-12,CUST1000,Male,47,Electronics,4,30,120


In [4]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [7]:
df['Day'] = df['Date'].dt.day  # División de fecha por día
df['Month'] = df['Date'].dt.month # División de fecha por Mes

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    1000 non-null   int64         
 1   Date              1000 non-null   datetime64[ns]
 2   Customer ID       1000 non-null   object        
 3   Gender            1000 non-null   object        
 4   Age               1000 non-null   int64         
 5   Product Category  1000 non-null   object        
 6   Quantity          1000 non-null   int64         
 7   Price per Unit    1000 non-null   int64         
 8   Total Amount      1000 non-null   int64         
 9   Day               1000 non-null   int32         
 10  Month             1000 non-null   int32         
dtypes: datetime64[ns](1), int32(2), int64(5), object(3)
memory usage: 78.2+ KB


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb
import lightgbm as lgb

In [9]:
# Dividir target y eliminar columnas irrelevantes
X = df.drop(['Total Amount', 'Date','Customer ID'], axis=1)  # Eliminamos Date, pero mantenemos las nuevas columnas derivadas
y = df['Total Amount']


In [19]:
#Preparación de datos: Divide los datos en un conjunto de entrenamiento (70%) y un conjunto de prueba (30%).
# Dividir los datos en conjuntos de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 58)

In [17]:
# Definición de categorías numéricas y categoricas
numeric_features = ['Age', 'Quantity', 'Price per Unit', 'Day','Month']  # Agrega las nuevas columnas de fecha si son útiles
categorical_features = ['Gender', 'Product Category']

In [12]:
# creación de un Pipeline para columnas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'mean')), # relleno de faltantes si es que los hay
    ('scaler', StandardScaler()) # Escalado de las categorías numéricas
])

In [15]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('nominal', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding de las categóricas
])

In [18]:
# Usar ColumnTransformer para aplicar las transformaciones a las columnas correctas
preprocessor = ColumnTransformer(
    transformers=[
        ('numerica', numeric_transformer, numeric_features),
        ('categorica', categorical_transformer, categorical_features)
    ],
   remainder='passthrough', # Dejar columnas no transformadas
   n_jobs= -1,
    verbose= True,
    sparse_threshold= 0.3
)

preprocessor

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('numerica',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Age', 'Quantity', 'Price per Unit', 'Day',
                                  'Month']),
                                ('categorica',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('nominal',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Gender', 'Product Category'])],
                  verbose=True)

In [35]:
pip install lazypredict

In [36]:
from lazypredict.Supervised import LazyRegressor
lazy = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None,)
models, predictions = lazy.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:07<00:00,  5.98it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000437 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 700, number of used features: 11
[LightGBM] [Info] Start training from score 458.950000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
DecisionTreeRegressor,1.00,1.00,0.00,0.08
RandomForestRegressor,1.00,1.00,0.00,0.23
BaggingRegressor,1.00,1.00,0.00,0.32
ExtraTreeRegressor,1.00,1.00,0.00,0.11
ExtraTreesRegressor,1.00,1.00,0.00,0.66
XGBRegressor,1.00,1.00,0.00,0.06
LGBMRegressor,1.00,1.00,0.01,0.16
HistGradientBoostingRegressor,1.00,1.00,0.01,0.72
GradientBoostingRegressor,1.00,1.00,2.85,0.58


In [21]:
#Selección de mejores hiperparámetros para cada modelo
M = {
    'Lineal': {
        'modelo': LinearRegression(),
        'parametros': {}

    },
    'modelo_KNN': {
        'modelo': KNeighborsRegressor(),
        'parametros': {
            'modelo__n_neighbors': [5, 10, 15],
            'modelo__weights': ['uniform', 'distance']
        }
    },
    'modelo_Arbol': {
        'modelo': DecisionTreeRegressor(),
        'parametros': {
            'modelo__max_depth': [None, 10, 20, 30],
            'modelo__min_samples_split': range(2, 21),
            'modelo__min_samples_leaf': range(1, 11)
        }
    },
    'modelo_RF': {
        'modelo': RandomForestRegressor(),
        'parametros': {
            'modelo__n_estimators': [50, 100],
            'modelo__max_depth': [None, 10, 20]
        }
    },
    'modelo_xgboost': {
        'modelo' : xgb.XGBRegressor(objective='reg:squarederror'),
        'parametros': {
        'modelo__n_estimators': [100, 200],
        'modelo__max_depth': [3, 6, 9]
        }
}
    }


In [24]:
resultados = {}
#Proceso de modelos
for modelo_nombre, config in M.items():
  modelo = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('modelo', config['modelo'])
])
  #Se realiza validación cruzada
  grid = GridSearchCV(modelo, config['parametros'], cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
  grid.fit(X_train, y_train)

  #Selección de los mejores parámetros
  mejores_parametros = grid.best_params_
  mejores_score = grid.best_score_
  mejor_modelo = grid.best_estimator_


 #Evaluación de los modelos con test.
  y_pred = mejor_modelo.predict(X_test)
  mse_test = mean_squared_error(y_test, y_pred)

  #Resultados
  resultados[modelo_nombre] = {
      'mejor_parametro': mejores_parametros,
      'mejor_score' : mejores_score,
      'mejor_modelo' : mejor_modelo
  }



In [26]:
optimizado = pd.DataFrame(resultados)
optimizado['modelo_Arbol']


,modelo_Arbol
mejor_parametro,"{'modelo__max_depth': None, 'modelo__min_sampl..."
mejor_score,0.0
mejor_modelo,"(ColumnTransformer(n_jobs=-1, remainder='passt..."


Se puede pareciar que el modelo árbol de decisión y Random Forest tiene muy buenas mpetricas y un R2 igual a 1. Es un modelo sin errores.

In [27]:
arbol = resultados['modelo_Arbol']
arbol = arbol['mejor_parametro']
arbol

{'modelo__max_depth': None,
 'modelo__min_samples_leaf': 1,
 'modelo__min_samples_split': 2}

In [28]:
#Modelo a optimizar
Arbol_D = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('modelo', DecisionTreeRegressor(max_depth=arbol['modelo__max_depth'],
                                     min_samples_leaf=arbol['modelo__min_samples_leaf'],
                                     min_samples_split=arbol['modelo__min_samples_split'],
                                     random_state=28))
])

#entrenamiento del modelo Árbol de Decisión
Arbol_D.fit(X_train,y_train)
#prediccion
y_pred_AD = Arbol_D.predict(X_test)

In [29]:
# Métricas de evaluación

#Evaluación del modelo: Evalúa el modelo utilizando el conjunto de prueba.
# Calcula el error cuadrático medio (MSE) y el R-cuadrado (R²) del modelo.

mse_A = mean_squared_error(y_test, y_pred_AD)
r2_A = r2_score(y_test, y_pred_AD)
mae_A = mean_absolute_error(y_test, y_pred_AD)

print(f'Error Cuadrático Medio: {mse_A}')
print(f'R-cuadrado: {r2_A}')
print(f'Error Absoluto Medio: {mae_A}')

Error Cuadrático Medio: 0.0
R-cuadrado: 1.0
Error Absoluto Medio: 0.0


Se puede apreciar que el modelo árbol de decisión no presenta errores y que su R2 es igual a 1.0

In [38]:
import joblib

# Guardar el preprocesador
joblib.dump(preprocessor,'ventas_retail_preproceso.pkl')

['ventas_retail_preproceso.pkl']